<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3041d1a92be0a1b3e2a78b3cbe7b116a03c0c639d95d6507090c2e799b5c74e0
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-19 12:28:08
-------------------
qualified stocks: 88
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.60 K
Current:  1.45 C
-------------------
Today PnL: 15.61 K (0.11%)
Current PnL: -20.85 L (-13.66%)
CY Booked + Current PnL: -6.89 L (-4.51%)
-------------------
Total profit:  1.91 L
Total loss:  -22.76 L
-------------------
Total Booked + Current PnL: 20.09 L (15.99%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.63%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.35 C
Est FTT PnL: 90.19 L (62.2%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.26%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,147.21,80.0,M-SC,1.91,162248.0,-12311.0,12347.0,-1.32,-7.05,7.61,0.02,234.0,-1.00,1.12,41.13,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,101.07,50.0,M-SC,3.73,86617.0,-14160.0,14248.0,0.20,-14.05,16.45,0.09,253.0,-0.99,0.60,13.80,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-14.93,48.0,X-MC,3.85,88800.0,-1878.0,16783.0,-0.22,-2.07,18.90,16.44,63.0,-0.11,0.61,17.69,X40,ATH,PHARMA
35,ICICIGI,2252.93,-14.37,59.0,X-MC,7.60,190362.0,16042.0,19169.0,1.04,9.20,10.07,20.19,68.0,0.84,1.31,25.71,X40,ATH,INSURANCE
84,VOLTAS,1530.00,-1.08,56.0,X-MC,3.72,210045.0,18303.0,19450.0,1.16,9.55,9.26,19.69,78.0,0.94,1.45,16.99,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1922.01,12.76,77.0,X-LC,4.38,261060.0,19136.0,40699.0,4.24,7.91,15.59,24.73,7.0,0.47,1.80,24.35,X40,ATH,IT
50,LTIM,7201.88,5.14,68.0,H-LC,11.46,269460.0,23309.0,54620.0,4.03,9.47,20.27,31.66,43.0,0.43,1.86,49.51,X200,ATH,IT
13,BSOFT,831.70,-8.39,62.0,H-SC,8.20,100585.0,-40068.0,111498.0,3.84,-28.49,110.85,50.79,151.0,-0.36,0.69,17.11,XR,ATH,IT
41,INFY,2275.00,-18.52,59.0,X-LC,3.62,328020.0,15132.0,156564.0,3.61,4.84,47.73,54.87,2.0,0.10,2.26,13.52,X40,BTT,IT
69,SONACOMS,806.63,-31.56,69.0,M-SC,6.78,88454.0,-12705.0,52710.0,3.46,-12.56,59.59,39.54,220.0,-0.24,0.61,24.73,AR,ATH,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,104.33,60.0,H-SC,16.03,139069.0,-79427.0,107625.0,-3.56,-36.35,77.39,12.90,161.0,-0.74,0.96,15.63,OX40N,NTT,FINANCE
64,SATIN,274.00,-25.10,52.0,H-SC,2.50,232712.0,-43572.0,186775.0,-2.24,-15.77,80.26,51.83,146.0,-0.23,1.61,13.37,XY24,NTT,FINANCE
51,MASFIN,398.61,-17.80,52.0,H-SC,7.18,93210.0,-4770.0,26369.0,-2.08,-4.87,28.29,22.05,164.0,-0.18,0.64,35.49,XR,ATH,FINANCE
63,SAMMAANCAP,326.00,19.75,54.0,M-SC,36.62,161298.0,12078.0,132103.0,-1.94,8.09,81.90,96.62,245.0,0.09,1.11,74.56,XY25,NTT,FINANCE
61,REPCOHOME,880.00,-55.29,48.0,H-SC,10.83,254709.0,-31796.0,289120.0,-1.89,-11.10,113.51,89.82,160.0,-0.11,1.76,34.76,XY24,NTT,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,STARHEALTH,761.0,36.05,66.0,H-SC,15.61,280214.0,-909.0,133774.0,-0.09,-0.32,47.74,47.26,171.0,-0.01,1.93,50.13,XY24,NTT,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-20.55,41.0,M-SC,10.14,144336.0,-3308.0,162638.0,-1.69,-2.24,112.68,107.91,216.0,-0.02,1.00,0.69,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,147.21,80.0,M-SC,1.91,162248.0,-12311.0,12347.0,-1.32,-7.05,7.61,0.02,234.0,-1.00,1.12,41.13,OX40N,NTT,PAINTS
79,TTKPRESTIG,770.00,101.07,50.0,M-SC,3.73,86617.0,-14160.0,14248.0,0.20,-14.05,16.45,0.09,253.0,-0.99,0.60,13.80,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-66.37,44.0,H-SC,1.99,220671.0,-48996.0,85333.0,-0.72,-18.17,38.67,13.47,158.0,-0.57,1.52,12.20,XY24,NTT,PAINTS
74,TATAELXSI,9161.00,-16.04,51.0,H-SC,6.32,102258.0,-29577.0,71806.0,1.99,-22.43,70.22,32.03,148.0,-0.41,0.71,14.54,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAMMAANCAP,326.00,19.75,54.0,M-SC,36.62,161298.0,12078.0,132103.0,-1.94,8.09,81.90,96.62,245.0,0.09,1.11,74.56,XY25,NTT,FINANCE
86,WIPRO,420.00,-13.34,56.0,M-LC,5.11,152861.0,1916.0,107538.0,2.35,1.27,70.35,72.51,99.0,0.02,1.05,7.37,XR,NTT,IT
38,INDIAMART,4810.62,-48.72,44.0,H-SC,4.56,127746.0,4410.0,127222.0,-1.47,3.58,99.59,106.72,139.0,0.03,0.88,27.79,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-48.72,44.0,H-SC,4.56,127746.0,4410.0,127222.0,-1.47,3.58,99.59,106.72,139.0,0.03,0.88,27.79,AR,ATH,MISC
86,WIPRO,420.00,-13.34,56.0,M-LC,5.11,152861.0,1916.0,107538.0,2.35,1.27,70.35,72.51,99.0,0.02,1.05,7.37,XR,NTT,IT
25,FINCABLES,1641.55,-20.55,41.0,M-SC,10.14,144336.0,-3308.0,162638.0,-1.69,-2.24,112.68,107.91,216.0,-0.02,1.00,0.69,OX40N,ATH,CABLES
87,ZYDUSLIFE,1286.17,-15.41,36.0,H-MC,5.25,206502.0,-3072.0,77748.0,0.65,-1.47,37.65,35.63,119.0,-0.04,1.42,14.96,AR,ATH,PHARMA
85,WHIRLPOOL,2270.00,-43.39,35.0,M-SC,7.83,110943.0,-3855.0,97896.0,-0.08,-3.36,88.24,81.92,236.0,-0.04,0.77,31.34,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-23.08,22.0,X-LC,0.26,238682.0,-37728.0,458890.0,-1.86,-13.65,192.26,152.37,3.0,-0.08,1.65,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,9.89,24.0,X-SC,10.90,83830.0,-44840.0,90134.0,-0.47,-34.85,107.52,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,-26.71,27.0,X-SC,1.32,104250.0,-23688.0,70817.0,-0.23,-18.52,67.93,36.84,90.0,-0.33,0.72,6.73,X40N,ATH,ELECTRICAL
26,GILLETTE,11206.78,-19.22,28.0,X-SC,6.11,249300.0,-4596.0,86906.0,-0.40,-1.81,34.86,32.42,85.0,-0.05,1.72,13.02,X40,ATH,FMCG
56,QUESS,424.00,-54.27,30.0,X-SC,37.28,45662.0,-19344.0,47196.0,-1.06,-29.76,103.36,42.84,83.0,-0.41,0.31,0.00,XY24,NTT,MISC


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TMPV,1065.00,-23.08,22.0,X-LC,0.26,238682.0,-37728.0,458890.0,-1.86,-13.65,192.26,152.37,3.0,-0.08,1.65,0.00,XY24,NTT,AUTO
43,ITC,452.00,-41.15,41.0,X-LC,0.92,197312.0,-2826.0,23717.0,-0.58,-1.41,12.02,10.44,5.0,-0.12,1.36,4.07,X40,NTT,FMCG
34,HONAUT,58357.33,-26.71,27.0,X-SC,1.32,104250.0,-23688.0,70817.0,-0.23,-18.52,67.93,36.84,90.0,-0.33,0.72,6.73,X40N,ATH,ELECTRICAL
36,ICICIPRULI,790.00,-21.49,56.0,X-MC,1.45,181514.0,4776.0,49953.0,-1.16,2.70,27.52,30.97,75.0,0.10,1.25,15.60,X40,ATH,INSURANCE
67,SIEMENS,4671.50,-1.60,58.0,H-LC,1.77,160350.0,-25745.0,73232.0,-1.31,-13.83,45.67,25.51,49.0,-0.35,1.11,18.17,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,9.89,24.0,X-SC,10.90,83830.0,-44840.0,90134.0,-0.47,-34.85,107.52,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
56,QUESS,424.00,-54.27,30.0,X-SC,37.28,45662.0,-19344.0,47196.0,-1.06,-29.76,103.36,42.84,83.0,-0.41,0.31,0.00,XY24,NTT,MISC
54,PAGEIND,51605.08,-29.94,33.0,X-MC,8.18,155680.0,-8060.0,50736.0,-0.54,-4.92,32.59,26.07,55.0,-0.16,1.07,0.00,X40,ATH,APPARELS
77,TMPV,1065.00,-23.08,22.0,X-LC,0.26,238682.0,-37728.0,458890.0,-1.86,-13.65,192.26,152.37,3.0,-0.08,1.65,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,-19.44,30.0,X-MC,6.93,174004.0,-26502.0,123665.0,-0.45,-13.22,71.07,48.45,64.0,-0.21,1.20,0.34,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-54.27,30.0,X-SC,37.28,45662.0,-19344.0,47196.0,-1.06,-29.76,103.36,42.84,83.0,-0.41,0.31,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-42.41,34.0,X-SC,3.11,78528.0,-66632.0,142560.0,-0.88,-45.90,181.54,52.31,91.0,-0.47,0.54,5.11,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,9.89,24.0,X-SC,10.90,83830.0,-44840.0,90134.0,-0.47,-34.85,107.52,35.20,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-14.93,48.0,X-MC,3.85,88800.0,-1878.0,16783.0,-0.22,-2.07,18.90,16.44,63.0,-0.11,0.61,17.69,X40,ATH,PHARMA
34,HONAUT,58357.33,-26.71,27.0,X-SC,1.32,104250.0,-23688.0,70817.0,-0.23,-18.52,67.93,36.84,90.0,-0.33,0.72,6.73,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4389.96,-25.57,63.0,X-LC,8.12,295771.0,-50185.0,116889.0,1.92,-14.51,39.52,19.28,1.0,-0.43,2.04,9.34,X40,ATH,IT
41,INFY,2275.00,-18.52,59.0,X-LC,3.62,328020.0,15132.0,156564.0,3.61,4.84,47.73,54.87,2.0,0.10,2.26,13.52,X40,BTT,IT
77,TMPV,1065.00,-23.08,22.0,X-LC,0.26,238682.0,-37728.0,458890.0,-1.86,-13.65,192.26,152.37,3.0,-0.08,1.65,0.00,XY24,NTT,AUTO
83,VBL,671.64,-19.21,44.0,X-LC,5.92,289333.0,-26509.0,139169.0,-0.96,-8.39,48.10,35.67,4.0,-0.19,2.00,4.30,X40N,ATH,FMCG
43,ITC,452.00,-41.15,41.0,X-LC,0.92,197312.0,-2826.0,23717.0,-0.58,-1.41,12.02,10.44,5.0,-0.12,1.36,4.07,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,7572.22,61.0,L-SC,18.01,84206.0,-9604.0,86219.0,-1.84,-10.24,102.39,81.67,271.0,-0.11,0.58,63.07,XR,NTT,CERAMICS
69,SONACOMS,806.63,-31.56,69.0,M-SC,6.78,88454.0,-12705.0,52710.0,3.46,-12.56,59.59,39.54,220.0,-0.24,0.61,24.73,AR,ATH,AUTO
51,MASFIN,398.61,-17.80,52.0,H-SC,7.18,93210.0,-4770.0,26369.0,-2.08,-4.87,28.29,22.05,164.0,-0.18,0.64,35.49,XR,ATH,FINANCE
71,SURYODAY,216.00,53.51,45.0,H-SC,12.65,141556.0,-37515.0,78762.0,0.47,-20.95,55.64,23.03,167.0,-0.48,0.98,40.45,XR,NTT,BANKS
81,VAIBHAVGBL,521.00,63.64,49.0,H-SC,10.77,142100.0,-40675.0,151749.0,-1.70,-22.25,106.79,60.77,159.0,-0.27,0.98,27.97,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,147.21,80.0,M-SC,1.91,162248.0,-12311.0,12347.0,-1.32,-7.05,7.61,0.02,234.0,-1.00,1.12,41.13,OX40N,NTT,PAINTS
13,BSOFT,831.70,-8.39,62.0,H-SC,8.20,100585.0,-40068.0,111498.0,3.84,-28.49,110.85,50.79,151.0,-0.36,0.69,17.11,XR,ATH,IT
5,ASIANPAINT,3460.25,0.77,84.0,X-LC,10.89,256432.0,4664.0,48081.0,0.28,1.85,18.75,20.95,31.0,0.10,1.77,38.15,X40,ATH,PAINTS
81,VAIBHAVGBL,521.00,63.64,49.0,H-SC,10.77,142100.0,-40675.0,151749.0,-1.70,-22.25,106.79,60.77,159.0,-0.27,0.98,27.97,XR,NTT,JEWELLERY
32,HCLTECH,1922.01,12.76,77.0,X-LC,4.38,261060.0,19136.0,40699.0,4.24,7.91,15.59,24.73,7.0,0.47,1.80,24.35,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.65
1,25,44.45
2,50,75.17


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.56
MC    28.64
LC    24.80
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.91
X40      20.61
X40N     11.49
XY25     10.12
XR        9.47
AR        9.04
OX40N     7.68
X200      1.86
SR        1.01
MH        0.81
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.23
X-MC    22.26
X-LC    19.81
M-SC    13.24
X-SC     7.69
H-MC     4.69
H-LC     2.97
L-SC     1.40
M-MC     1.36
M-LC     1.05
L-LC     0.97
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.58,-4.65,38.49
IT,13.61,-12.85,71.16
FINANCE,10.45,-12.20,64.97
MISC,7.35,-21.57,71.38
PAINTS,6.05,-5.20,20.82
ELECTRICAL,5.62,-11.12,50.51
INSURANCE,4.49,3.05,31.11
PHARMA,3.42,-0.57,34.32
AUTO,3.35,-18.62,132.34


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3275312.0,21
AR,1260043.0,10
XR,1256672.0,13
X40,978829.0,14
X40N,705105.0,9
OX40N,682116.0,10
XY25,465207.0,7
SR,275113.0,2
MH,66536.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3467037.0,25
M-SC,1435825.0,16
X-LC,1194980.0,11
X-MC,1163579.0,16
X-SC,725843.0,8
H-MC,398997.0,3
L-SC,257265.0,3
H-LC,127852.0,2
M-LC,107538.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1177464.0      6
           AR         849954.0      5
           XR         779294.0      7
M-SC       XY24       760386.0      6
X-LC       XY24       529273.0      2
X-MC       X40        415839.0      7
X-LC       X40        385950.0      5
X-MC       XY24       343386.0      3
H-SC       OX40N      318676.0      4
X-SC       XY24       281074.0      3
M-SC       OX40N      275935.0      5
H-SC       SR         275113.0      2
X-SC       X40N       267729.0      3
X-MC       X40N       229960.0      4
H-MC       AR         215268.0      2
X-LC       X40N       207416.0      2
H-MC       XY24       183729.0      1
X-SC       X40        177040.0      2
X-MC       XY25       174394.0      2
L-SC       XR         169760.0      2
M-SC       XR         145812.0      2
           XY25       132103.0      1
           AR         121589.0      2
M-LC       XR         107538.0      1
L-SC       OX40N       87505.0      1
H-LC       AR          73232.0      1
X-LC       XY25        72341.0      2
H-SC       MH          66536.0      1
H-LC       X200        54620.0      1
L-MC       XR          54268.0      1
M-MC       XY25        48019.0      1
L-LC       XY25        38350.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 52.0 seconds
